<img src="../figs/eh_logo.png" style="width: 200px;">

# Getting started with EmptyHeaded

This tutorial provides the basic information to get you up and running with EmptyHeaded. In this tutorial we show you how to make databases great again:

1. Provide a brief [system overview](#system_overview) 
2. Show how to [run a sample query](#run_a_query)

This example assumes that you have resolved all dependencies listed on our [GitHub](https://github.com/hazyresearch/EmptyHeaded) page and were able to  successfully setup the EmptyHeaded environment and compile the engine.

## Prerequisites

Before we go any further, make sure you have done the following **prior** to loading this notebook:

1. Run *source ./env.sh* in the root of the EmptyHeaded repository.
2. Run *./compile.sh* in the root of the EmptyHeaded repository.

<a id='system_overview'></a>

# System Overview

The EmptyHeaded engine works in three phases:
(1) the query compiler translates a high-level datalog-like query
language into a logical query plan represented as a
GHD, replacing relational algebra in its traditional role; (2) the query compiler
generates code for the execution engine by translating the GHD into a
series of set intersections and loops; and (3) the execution engine
performs automatic algorithmic and representation decisions based upon
skew in the data.

<br/>
<div style="text-align:left">
<img src="../figs/systemOverview.png" style="width: 550px">
</div>

1. **Query Compiler**: our query compiler is written in Scala and the code can be found [here](https://github.com/HazyResearch/EmptyHeaded/tree/master/query_compiler).
2. **Code Generation**: our code generator is written in Scala and the code can be found [here](https://github.com/HazyResearch/EmptyHeaded/tree/master/query_compiler).
3. **Storage Engine**: our storage engine is written in C++ and the code can be found [here](https://github.com/HazyResearch/EmptyHeaded/tree/master/storage_engine).

<a id='run_a_query'></a>

# Run a Query

We now show how to run a query in the EmptyHeaded engine. There are 3 phases to running a query in the EmptyHeaded engine:

1. [Build a database](#build_db)
2. [Execute a query](#execute)
3. [Inspect output data.](#inspect)

We overview each phase next.

<a id='build_db'></a>
## Build a database

The easiest way to get data in and out of the EmptyHeaded engine is via Pandas DataFrames. Note: we currently only accept DataFrames that contain numeric types but we show in our [rdf tutorial](https://github.com/HazyResearch/EmptyHeaded/tree/master/docs/notebooks/rdf) how one can bypass using Pandas and load data directly into the EmptyHeaded engine (for things like strings).

The process of building a EmptyHeaded database is the process of loading tables, dictionary encoding these tables, and translating these tables into Tries which are our core data structure. We then write these tries to disk in a binary format which can be loaded as needed for query exection. A visual description of this process is below. 

<br/>
<img src="../figs/table_transform.png" style="width: 550px">

Ok, enough talking, let's start running! First import the emptyheaded python module.

In [1]:
from emptyheaded import *

Our query compiler is written in Scala. We spin up the JVM with the following command.

In [2]:
start()

You can change how many threads we run with, etc., via the configuration that will be used for the database instance.

In [3]:
c = Config(num_threads=4)
print c

(system: emptyheaded, num_threads: 4,
		num_sockets: 4, layout: hybrid, memory: RAM)


### Pandas DataFrames

We accept data in the form of a Pandas DataFrame. Lets [read some data from a csv file](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) into a Pandas DataFrame. This dataset is a (small) graph dataset from our [Snap](https://snap.stanford.edu/data/egonets-Facebook.html) friends. 

The accepted datatypes when transferring Pandas DataFrames into the EmptyHeaded engine are:

- np.int32,
- np.int64,
- np.uint32,
- np.uint64,
- np.float32,
- np.float64

In [4]:
eh = os.path.expandvars("$EMPTYHEADED_HOME")
df_graph = pd.read_csv(eh+'/test/graph/data/facebook_duplicated.tsv',\
  sep='\t',\
  names=["0","1"],\
  dtype={"0":np.uint32,"1":np.uint32})

Now, let's create a relation. A relation has a name and takes a dataframe that contains the data. The schema is automatically inferred.

In [5]:
edge = Relation(
  name="Edge",
  dataframe=df_graph)

We are finally ready to create a database. The database accepts a config (we created one above), a path to the folder where the database will be placed on disk (must not exist already), and a list of relations that the database will contain. This step is slow in comparison to the others as it compiles the backend for the config requirements specified, performs dictionary encoding, builds tries, and writes tries to disk.

In [6]:
#add one more relation we need for other tutorials
deg = pd.read_csv(eh+"/test/graph/data/inv_degree.tsv",\
  sep='\t',\
  names=["0","a_0"],\
  dtype={"0":np.uint32,"a_0":np.float32})
inv_degree = Relation(
    name="InvDegree",
    dataframe=deg)
db = Database.create(
  c,
  eh+"/docs/notebooks/graph/db",
  [edge,inv_degree])
db.build()

java.lang.ExceptionPyRaisable: java.lang.Exception: ERROR DATABASE FOLDER EXISTS OR IS AN INVALID PATH.

That step was slow, it would be nice if we could avoid it if we already have created this database. We can do that with the following command.

In [7]:
db = Database.from_existing(eh+"/docs/notebooks/graph/db")

Success! You have created your first EmptyHeaded database. Time to run a query!

<a id='execute'></a>
## Execute a Query

Now that a database is loaded we are ready to execute a query over the database. We provide details on our query language and accepted datatypes in our [query language tutorial](https://github.com/HazyResearch/EmptyHeaded/tree/master/docs/notebooks). For now we will show how to find all the triangles in this dataset. Execute the datalog rule below to do this! Note: if you are benchmarking EmptyHeaded we print the time to compute each portion of the query in the shell that you launched iPython notebooks from. This separates data loading time from actual query execution! We talk about how to look at the generated code in our [storage engine tutorial](https://github.com/HazyResearch/EmptyHeaded/tree/master/docs/notebooks).

In [8]:
db.eval("Triangle(a,b,c) :- Edge(a,b),Edge(b,c),Edge(a,c).")

<a id='inspect'></a>
## Inspect Ouput

Now that a query has been executed let's look at the output of the query. We can return it to the front-end as follows. Note this is just a wrapper around the back-end class. We can inspect the following fields of the relation.

In [9]:
tri = db.get("Triangle")
print "is annotated: " + str(tri.annotated)
print "number of triangles: " +str(tri.num_rows)
print "number of columns:" + str(tri.num_columns)

is annotated: False
number of triangles: 9672060
number of columns:3


We can return a dataframe of the relation with the following command.

In [10]:
print tri.getDF()

            0     1     2
0           0     1    48
1           0     1    53
2           0     1    54
3           0     1    73
4           0     1    88
5           0     1    92
6           0     1   119
7           0     1   126
8           0     1   133
9           0     1   194
10          0     1   236
11          0     1   280
12          0     1   299
13          0     1   315
14          0     1   322
15          0     1   346
16          0     2    20
17          0     2   115
18          0     2   116
19          0     2   149
20          0     2   226
21          0     2   312
22          0     2   326
23          0     2   333
24          0     2   343
25          0     3     9
26          0     3    25
27          0     3    26
28          0     3    67
29          0     3    72
...       ...   ...   ...
9672030  4038  4004  3980
9672031  4038  4004  4013
9672032  4038  4004  4020
9672033  4038  4004  4023
9672034  4038  4004  4031
9672035  4038  4013  3980
9672036  403

# Spinning Down

We stop the JVM with the following command.

In [11]:
stop()